# Automatas Celulares Neuronales

Este Jupyter Notebook presenta el comportamiento de diferentes automatas celulares neuronales. Cada kernel se encuentra en la carpeta *Kernels*, las funciones de activación necesarias se codifican en cada punto.

## Bibliotecas necesarias y configuración inicial

In [1]:
import cv2
from IPython.display import Video, display
import json
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from scipy.signal import convolve2d
from tqdm import tqdm
from inspect import getsource

In [2]:
plt.rc('axes', axisbelow=True)
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.rcParams['animation.embed_limit'] = 4**128
plt.ioff();

## Funciones a utilizar

In [3]:
def create_animation(frame, kernel, activation_function, N, persistent=False):
    # Create an array to store all the frames
    frames = np.empty((N,*frame.shape))
    # Create the next n frames
    for i in tqdm(range(N), desc='Applying convolutions'):
        frames[i] = frame
        # Convolve an apply the activation function to a frame
        frame = convolve2d(frame, kernel, mode='same', boundary='wrap')
        frame = activation_function(frame)
        # Keep values of the frame n-1
        if persistent and i > 0:
            frame = frame + frames[i-1]
        # Clip the values in an array.
        frame[frame<0] = 0
        frame[frame>1] = 1
    return frames

def export_video_cv2(frames, name='output.mp4', fps = 25, skip_frames=False, skip='even'):
    # Get the size of every dimension
    duration, height, weight = frames.shape
    if skip_frames:
        k = 0 if skip == 'even' else 1 if 'odd' else skip
    # Create writer
    out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (weight, height), False)
    # Create video
    for i in tqdm(range(duration), desc='Creating video'):
        if skip_frames:
            if i%2 != k:
                data = (frames[i]*255).astype('uint8')
                out.write(data)
        else:
            data = (frames[i]*255).astype('uint8')
            out.write(data)
    out.release()
    
def export_video_mat(frames, name='output.mp4', figsize=(16,9), fps=25, 
                     interval=20, skip_frames=False, skip='even', cmap='magma'):
    
    N = frames.shape[0]
    
    fig, ax = plt.subplots(figsize=figsize)
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    im = ax.imshow(frames[0], cmap=cmap)
    
    if skip_frames:
        k = 0 if skip == 'even' else 1 if 'odd' else skip
        
    def animate(i):
        if skip_frames: 
            if i%2 != k:
                im.set_array(frames[i])
        else:
            im.set_array(frames[i])
    plt.close()

    anim = animation.FuncAnimation(fig, animate, frames=N, interval=interval)
    writervideo = animation.FFMpegWriter(fps=fps) 
    anim.save(name, writer=writervideo)
    

def get_kernel(path):
    with open(path, 'r') as file:
        data = json.load(file)
        kernel = np.zeros(9)
        for key,value in data['filter'].items():
            kernel[int(key)] = value

    return kernel.reshape(3,3)

## Resultados

### Juego de la vida

In [4]:
# Size
weight = 640
height = 360
# Number of iterations
N = 5000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame = np.random.randint(0,2, size=(height,weight))
# Select kernel
kernel = get_kernel('Kernels/game_of_life.json')
# Select activation function
activation_function = lambda x: np.where((x == 3)|(x == 11)|(x == 12), 1, 0)
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
#export_video_cv2(frames, name='Videos/game_of_life.mp4', fps=60, skip_frames=True, skip='odd')
export_video_mat(frames, name='Videos/game_of_life.mp4', fps=60, cmap='binary', skip_frames=True, skip='odd')
del frames

Applying convolutions: 100%|███████████████| 5000/5000 [00:28<00:00, 176.95it/s]


In [5]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/game_of_life.mp4', html_attributes='loop autoplay', height=500)


kernel = 
[[1. 1. 1.]
 [1. 9. 1.]
 [1. 1. 1.]]


activation_function = lambda x: np.where((x == 3)|(x == 11)|(x == 12), 1, 0)




[Juego de la vida (video)](https://youtu.be/x-yBPAsFn8g?t=0&si=1071c77Hb4UNQNEg)

### Regla 30

In [6]:
# Size
weight = 640
height = 360
# Number of iterations
N = 350
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame[0,weight//2] = 1
# Select kernel
kernel = get_kernel('Kernels/rule30.json')
# Select activation function
activation_function = lambda x: np.where((x == 1)|(x == 2)|(x == 3)|(x == 4), 1, 0)
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=True)
#export_video_cv2(frames, name='example.mp4', fps=60)
export_video_mat(frames, name='Videos/rule30.mp4', fps=60, cmap='gray')

Applying convolutions: 100%|█████████████████| 350/350 [00:02<00:00, 162.59it/s]


In [7]:
print(f'\nkernel = \n{kernel[-1]}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/rule30.mp4', html_attributes='loop autoplay', height=500)


kernel = 
[1. 2. 4.]


activation_function = lambda x: np.where((x == 1)|(x == 2)|(x == 3)|(x == 4), 1, 0)




[Regla 30 (video)](https://youtu.be/x-yBPAsFn8g?t=15&si=N_Q8h7-Hb352bcVB)

### Moho 1

In [8]:
# Size
weight = 640
height = 360
# Number of iterations
N = 2000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame[3*height//4,weight//3] = 1
init_frame[height//4,2*weight//3] = 1
init_frame[5*height//6,5*weight//6] = 1
init_frame[5*height//6+1,5*weight//6] = 1
init_frame[5*height//6,5*weight//6+1] = 1
# Select kernel
kernel = get_kernel('Kernels/moho1.json')
# Select activation function
activation_function = lambda x: np.sin(np.abs(x/2))
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
#export_video_cv2(frames, name='Videos/moho1.mp4', fps=60, skip_frames=True, skip='odd')
export_video_mat(frames, name='Videos/moho1.mp4', fps=60, cmap='magma', skip_frames=True, skip='odd')
del frames

Applying convolutions: 100%|███████████████| 2000/2000 [00:12<00:00, 153.95it/s]


In [9]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/moho1.mp4', html_attributes='loop autoplay', height=500)


kernel = 
[[-0.84899998  0.912      -0.84899998]
 [ 0.912       0.          0.912     ]
 [-0.84899998  0.912      -0.84899998]]


activation_function = lambda x: np.sin(np.abs(x/2))




[Moho 1 (video)](https://youtu.be/x-yBPAsFn8g?t=20&si=QNrFRbTPXUKE1_NL)

### Moho 2

In [10]:
# Size
weight = 640
height = 360
# Number of iterations
N = 2500
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame[height//2,weight//2] = 1
# Select kernel
kernel = get_kernel('Kernels/moho2.json')
# Select activation function
activation_function = lambda x: np.sin(np.abs(x/2))
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/moho2.mp4', fps=60, skip_frames=True, skip='odd')
del frames

Creating video: 100%|██████████████████████| 2500/2500 [00:04<00:00, 609.92it/s]


In [24]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/moho2.mp4', html_attributes='loop autoplay', height=500)


kernel = 
[[-0.93900001  0.88       -0.93900001]
 [ 0.88        0.40000001  0.88      ]
 [-0.93900001  0.88       -0.93900001]]


activation_function = lambda x: -1/(0.9*np.power(x, 2)+1)+1




[Moho 2 (video)](https://youtu.be/x-yBPAsFn8g?t=40&si=mrjD3eCzEok8iSbY)

### Gusanos 1

In [12]:
# Size
weight = 640
height = 360
# Number of iterations
N = 10000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame = np.random.uniform(0,1, size=(height,weight))
# Select kernel
kernel = get_kernel('Kernels/worms1.json')
# Select activation function
activation_function = lambda x: -1/(0.89*np.power(x, 2)+1)+1
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/worms1.mp4', fps=60, skip_frames=True, skip='odd')
del frames

Creating video: 100%|████████████████████| 10000/10000 [00:43<00:00, 230.52it/s]


In [13]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/worms1.mp4', html_attributes='controls', height=500)


kernel = 
[[ 0.80000001 -0.85000002  0.80000001]
 [-0.85000002 -0.2        -0.85000002]
 [ 0.80000001 -0.85000002  0.80000001]]


activation_function = lambda x: -1/(0.89*np.power(x, 2)+1)+1




[Gusanos 1 (video)](https://youtu.be/x-yBPAsFn8g?t=59&si=C65Qd6ldE771uc-f)

### Gusanos 2

In [14]:
# Size
weight = 640
height = 360
# Number of iterations
N = 5000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame[height//2,weight//2] = 0.0005
# Select kernel
kernel = get_kernel('Kernels/worms2.json')
# Select activation function
activation_function = lambda x: np.sin(np.abs(x/2))
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/worms2.mp4', fps=60, skip_frames=True, skip='odd')
del frames

Creating video: 100%|██████████████████████| 5000/5000 [00:30<00:00, 163.66it/s]


In [15]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/worms2.mp4', html_attributes='controls', height=500)


kernel = 
[[-0.84899998  0.86299998 -0.84899998]
 [ 0.86299998  0.18799999  0.86299998]
 [-0.84899998  0.86299998 -0.84899998]]


activation_function = lambda x: np.sin(np.abs(x/2))




[Gusanos 2 (video)](https://youtu.be/x-yBPAsFn8g?t=80&si=Z5MnKl1o4LnrGqXz)

### Caminos

In [16]:
# Size
weight = 640
height = 360
# Number of iterations
N = 1000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame = np.random.uniform(0,1, size=(height,weight))
# Select kernel
kernel = get_kernel('Kernels/paths.json')
# Select activation function
gaussian = lambda x, b: 1/np.power(2,np.power(x-b, 2))
activation_function = lambda x: gaussian(x, 3.5)
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/paths.mp4', fps=60, skip_frames=True, skip='odd')
del frames

Creating video: 100%|█████████████████████| 1000/1000 [00:00<00:00, 1521.76it/s]


In [17]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
print(f'\n{getsource(gaussian)}\n')
#Video('Videos/paths.mp4', html_attributes='controls', height=500)


kernel = 
[[0. 1. 0.]
 [1. 1. 1.]
 [0. 1. 0.]]


activation_function = lambda x: gaussian(x, 3.5)



gaussian = lambda x, b: 1/np.power(2,np.power(x-b, 2))




[Caminos (video)](https://youtu.be/x-yBPAsFn8g?t=100&si=_30Hd67SEP2wlqMU)

### Olas

In [18]:
# Size
weight = 640
height = 360
# Number of iterations
N = 10000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame = np.random.uniform(0,1, size=(height,weight))
# Select kernel
kernel = get_kernel('Kernels/waves.json')
# Select activation function
activation_function = lambda x: np.abs(1.2*x)
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/waves.mp4', fps=30, skip_frames=True, skip='odd')
del frames

Creating video: 100%|████████████████████| 10000/10000 [01:15<00:00, 133.28it/s]


In [19]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/waves.mp4', html_attributes='controls', height=500)


kernel = 
[[ 0.56459999 -0.7159      0.56459999]
 [-0.7159      0.62690002 -0.7159    ]
 [ 0.56459999 -0.7159      0.56459999]]


activation_function = lambda x: np.abs(1.2*x)




[Olas (video)](https://youtu.be/x-yBPAsFn8g?t=121&si=kfN2SsDkMNv-exvB)

### Bacteria

In [20]:
# Size
weight = 640
height = 360
# Number of iterations
N = 10000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame[height//2,weight//2] = 0.2
init_frame[height//2-1,weight//2-1] = 1
# Select kernel
kernel = get_kernel('Kernels/bacteria.json')
# Select activation function
activation_function = lambda x: np.sin(np.abs(x/2))
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/bacteria.mp4', fps=60, skip_frames=True, skip='odd')
#export_video_mat(frames, name='Videos/bacteria.mp4', fps=60, cmap='RdPu')
del frames

Creating video: 100%|████████████████████| 10000/10000 [01:08<00:00, 145.31it/s]


In [21]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/bacteria.mp4', html_attributes='controls', height=500)


kernel = 
[[-0.89999998  0.94       -0.89999998]
 [ 0.94        0.          0.94      ]
 [-0.89999998  0.94       -0.89999998]]


activation_function = lambda x: np.sin(np.abs(x/2))




[Bacterias (video)](https://youtu.be/x-yBPAsFn8g?t=142&si=NabVdbii1_WGuT4i)

### Mitosis

In [22]:
# Size
weight = 640
height = 360
# Number of iterations
N = 5000
# Initial conditions
init_frame = np.zeros((height, weight))
init_frame = np.random.uniform(0,1, size=(height,weight))
# Select kernel
kernel = get_kernel('Kernels/mitosis.json')
# Select activation function
activation_function = lambda x: -1/(0.9*np.power(x, 2)+1)+1
# Compute the N iterations
frames = create_animation(init_frame, kernel, activation_function, N, persistent=False)
export_video_cv2(frames, name='Videos/mitosis.mp4', fps=30, skip_frames=True, skip='odd')
del frames

Creating video: 100%|██████████████████████| 5000/5000 [00:05<00:00, 837.27it/s]


In [23]:
print(f'\nkernel = \n{kernel}\n')
print(f'\n{getsource(activation_function)}\n')
#Video('Videos/mitosis.mp4', html_attributes='controls', height=500)


kernel = 
[[-0.93900001  0.88       -0.93900001]
 [ 0.88        0.40000001  0.88      ]
 [-0.93900001  0.88       -0.93900001]]


activation_function = lambda x: -1/(0.9*np.power(x, 2)+1)+1




[Mitosis (video)](https://youtu.be/x-yBPAsFn8g?t=163&si=ItamYLnMLLLsD6HF)